In [ ]:
# import importlib

# # # 1. 변경된 모듈을 평소처럼 import 합니다. (예: 'my_module.py'를 import)
# import db_handler 

# # # 2. 모듈의 내용이 변경되었다면, 아래 코드를 실행하여 다시 로드합니다.
# importlib.reload(db_handler)

<module 'db_handler' from 'c:\\SKN_19\\ZIP-FIT-2\\lab\\whatsup\\db_handler.py'>

In [5]:
from db_handler import DataBaseHandler

In [6]:
zf_db = DataBaseHandler()

with zf_db as cursor:
    cursor.execute("SELECT version();")
    print(cursor.fetchone())

('PostgreSQL 17.7 (Debian 17.7-3.pgdg12+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14+deb12u1) 12.2.0, 64-bit',)
